### Analyze Simulator Data

The first 20 attempts at the self-driving simulator yielded a single success, iteration #10. This attempt used a combination of 16,440 image samples (data provided, 5 laps of track 1, 1 lap of track 2), plus augmented samples based on splitting the viewport into three lateral views: center (main sample), left and right. The left and right viewports were assigned random steering values (normal distribution * 0.5) designed to move the car back to center. 

This attempt successfully navigated the first corner 9 times out of 10 but resulted in the car weaving back and forth across the road. It also resulted in the model plateauing at around 35% accuracy no matter how much the batch size, epochs were changed. Finally, it was never able to navigate the second corner. The weaving almost looked like the model was replicating the function used to derive the steering angles for the L and R viewports.

All subsequent iterations tried to build on this notion by augmenting the data both with up/down sampling and various algorithms designed to adjust the steering based on the viewport. None were successful at even navigating the first corner.

This led to a few conclusions:

- The fault lay in the data. There were not enough examples of the corner cases (pun intended) to teach the car in general how to navigate turns. Generating data was not the solution, new training data especially for corners was required. Initial sample analysis led to this same conclusion but was ignored because it was thought that augmentation/downsampling could adequately compensate.
- The classifier was inadequate to the task. It was a two-layer design trying to learn 201 classes (more on that next) from complex input data. Additional layers (at least 2 more were needed).
- The initial use of 201 classes for the feature space (-1.00 to +1.00) was deemed necessary for ultimate resolution in the steering angle output. However, upon furture examination of the steering data that number could be drastically reduced to between 21 and 51. 21 classes would give a resolution of +/-0.05 in the steering angle; 51 classes +/-0.02. This would also have the positive side effect of smoothing the car's motion and consolidating samples for steering. 
- The choice is 51 classes because this will result in a 1 degree steering resolution
- The analysis done previously was insufficient to ensure proper data distribution over various steering angles. New statistical analysis was needed. 
- The viewport methodology was a good but extreme way of trying to generalize the samples. Increasing the center viewport width to 2/3 of the image width would allow encompassing more information from the view while also allowing for a different upsampling techniqe: dithering through latteral translation while keeping the same steering angle. This would account for varying positions in the lane while avoiding the difficult task of deriving a new steering angle.


In [ ]:
import numpy as np
from numpy import genfromtxt
from ImageProcessing import data_binning
from TrainSimulator import *
import matplotlib.image as mpimg 

#Format probabilities in a readable fashion
float_formatter = lambda x: "%.3f" % x
np.set_printoptions(formatter={'float_kind':float_formatter})

n_classes = 51

file_format = [('center','S64'),('left','S64'),('right','S64'),('steering', 'f8'),('throttle', 'f8')]
combined = genfromtxt('data/driving_log.csv', dtype=file_format, delimiter=',', skip_header=0, usecols=(0, 1, 2, 3, 4))

#Scale steering and throttle
y_data = scale_labels(combined['steering'], n_classes)


In [ ]:
print(len(y_data))
#Augment with left and right cameras
cache = []
y_data_aug = None
mid = (n_classes-1)/2
correction = round(0.1 * mid)
for row in zip(y_data):
    l = row[0] + correction
    r = row[0] - correction
    if l > mid*2 or r < 0:
        print(correction, row[0], l, r)
    cache = cache + [l,r]

y_data_aug = np.array(cache)
y_data_aug = np.append(y_data, y_data_aug)
print(len(y_data_aug))

In [ ]:
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
%matplotlib inline

gs2 = gridspec.GridSpec(2, 3)
fig = plt.figure(figsize=(10,10))

com = plt.subplot(2,3,5)
o = plt.hist(y_data, bins=n_classes)
t = com.set_title("Combined")


In [ ]:
x_data,y_data = downsample(combined['center'], y_data, n_classes, 2000)

In [ ]:
sample_means = sample_logits(x_data)
print("Means \tRed \tGreen \tBlue")
print(sample_means[0],"\t",sample_means[1],"\t",sample_means[2])

In [ ]:
fig = plt.figure(1)
o = plt.hist(y_data, bins=n_classes)
#t = fig.set_title("After Downsampling")
print("New Mean", np.mean(y_data), "Median", np.median(y_data), "Count", len(y_data))

### Dataset Analysis

In [1]:
import imageio
imageio.plugins.ffmpeg.download()
import numpy as np
from numpy import genfromtxt
from moviepy.editor import VideoFileClip
import moviepy.editor as mpy
from IPython.display import HTML
import pickle
import cv2

file_format = [('center','S64'),('left','S64'),('right','S64'),('steering', 'f8'),('throttle', 'f8')]
dataset = genfromtxt('data/driving_log-flawed-Track2.csv', dtype=file_format, delimiter=',', skip_header=0, usecols=(0, 1, 2, 3, 4))

fps = 15
sequence_length = len(dataset)
sequence_start = 0
sequence_duration = int(sequence_length/fps)

def make_video(t):
    idx = sequence_start + int(t / (1/fps))
    filename = "./data/{0}".format(dataset['center'][idx].decode("utf-8").strip())
    image = cv2.imread(filename)
    label = "{0}".format(idx)
    cv2.putText(image, label,(20,20),cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255),thickness=2)
    return cv2.cvtColor(image.astype("uint8"), cv2.COLOR_BGR2RGB)

vid_output = "review_clip_track2.mp4"
clip = mpy.VideoClip(make_video, duration=sequence_duration)
%time clip.write_videofile(vid_output, fps=fps, audio=False)


[MoviePy] >>>> Building video review_clip_track2.mp4
[MoviePy] Writing video review_clip_track2.mp4


100%|████████████████████████████████████████████████████████████████████████████▉| 3960/3961 [00:10<00:00, 371.80it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: review_clip_track2.mp4 

Wall time: 10.8 s


In [ ]:
for i in range(sequence_length):
    filename = "./data/{0}".format(dataset['center'][i].decode("utf-8").strip())
    image = cv2.imread(filename)
    if image is None:
        print(filename)
